In [2]:
#------------------------------------------------------------------------------------------------------------
function Trapezium(f, a, b, n)
    h=(b-a)/n
    s = 0
    for i in 1:n
        s += 0.5 * (f(a + (i-1)*h) + f(a + i*h))
    end
    return h*s
end
#------------------------------------------------------------------------------------------------------------
function MC(f, A, B, C, D, N)
    count = 0
    for n = 1:N
        x,y = (B - rand()*(B-A)), (D - rand()*(D-C))
        if y > 0 && y <= f(x)
            count += 1
        elseif y < 0 && y >= f(x)
            count -= 1
        end
    end
    return (count/N)*((B-A)*(D-C))
end
#------------------------------------------------------------------------------------------------------------
function runge_kutta( f, h, y_0, t_0, t_1 )   
    
    #author: Harry Cai
    
t = t_0:h:t_1;
y = zeros(length(t));
y[1] = y_0;

for i = 1:length(t)-1
    k1 = f(t[i], y[i]);
    k2 = f(t[i]+h/2, y[i]+h*k1/2);
    k3 = f(t[i]+h/2, y[i]+h*k2/2);
    k4 = f(t[i]+h, y[i]+h*k3);
    y[i+1] = y[i] + (h/6) * (k1 + 2*k2 + 2*k3 + k4);
end

return y    
    
end

function rk_integration(f, a, b, n)
    steps = div(n, 4) # number of steps, which is n/4 as each step requires 4 evaluations of f
    F(t, y) = f(t);
    t = a:(b - a)/steps:b;
    h = t[2]-t[1];
    y = runge_kutta(F, h, 0, a, b);
    return y[steps];
end
#------------------------------------------------------------------------------------------------------------

using SpecialFunctions
using Suppressor
using Plots
plotlyjs()

runs = 1000

trErrors = zeros(runs)
trTimes = zeros(runs)

mcErrors = zeros(runs)
mcTimes = zeros(runs)

rkErrors = zeros(runs)
rkTimes = zeros(runs)

for i = 1 : runs
    
    scale = 10*rand()
    a = 10.0^(3*rand() - 4)
    b = 10.0^(3*rand())
    f(x) = scale*sin(1/x)
    
    @suppress tempTrueVal = scale*((b*sin(1/b) - cosint(1/b)) - (a*sin(1/a) - cosint(1/a)))
    
    tic()
    tempVal = Trapezium(f, a, b, 100)
    @suppress tempTime = toc()
    trErrors[i] = abs((tempTrueVal - tempVal)/tempTrueVal)
    trTimes[i] = tempTime
    
    tic()
    tempVal = MC(f, a, b, -scale, scale, 1000)
    @suppress tempTime = toc()
    mcErrors[i] = abs((tempTrueVal - tempVal)/tempTrueVal)
    mcTimes[i] = tempTime
    
    tic()
    tempVal = rk_integration(f, a, b, 100)
    @suppress tempTime = toc()
    rkErrors[i] = abs((tempTrueVal - tempVal)/tempTrueVal)
    rkTimes[i] = tempTime
end

trAvgError = mean(trErrors)
trVariance = var(trErrors)
trAvgTime = mean(trTimes)

mcAvgError = mean(mcErrors)
mcVariance = var(mcErrors)
mcAvgTime = mean(mcTimes)

rkAvgError = mean(rkErrors)
rkVariance = var(rkErrors)
rkAvgTime = mean(rkTimes)

println("Average Trapezium Rule Relative Error: ", trAvgError)
println("Trapezium Rule Variance: ", trVariance)
println("Average Trapezium Rule Time: ", trAvgTime)
println()
println("Average Monte Carlo Relative Error: ", mcAvgError)
println("Monte Carlo Variance: ", mcVariance)
println("Average Monte Carlo Time: ", mcAvgTime)
println()
println("Average Runge-Kutta Relative Error: ", rkAvgError)
println("Runge-Kutta Variance: ", rkVariance)
println("Average Runge-Kutta Time: ", rkAvgTime)

methods = ["Trapezium Rule", "Monte Carlo", "Runge-Kutta"]
errors = [trAvgError, mcAvgError, rkAvgError]
vars = [trVariance, mcVariance, rkVariance]
times = [trAvgTime, mcAvgTime, rkAvgTime]

fig1 = bar(methods,errors,title="Relative Error",xlabel="Method",ylabel="Relative Error",legend=false)
fig2 = bar(methods,vars,title="Variance",xlabel="Method",ylabel="Variance",legend=false)
fig3 = bar(methods,times,title="Running Times",xlabel="Method",ylabel="Running Time (s)",legend=false)

display(fig1)
display(fig2)
display(fig3)







Average Trapezium Rule Relative Error: 0.08782714582810133
Trapezium Rule Variance: 0.019010042887813026
Average Trapezium Rule Time: 6.296265999999999e-5

Average Monte Carlo Relative Error: 0.14628297015714276
Monte Carlo Variance: 0.02827787248345273
Average Monte Carlo Time: 0.00016730237199999998

Average Runge-Kutta Relative Error: 0.13261174452219823
Runge-Kutta Variance: 0.033407662488274294
Average Runge-Kutta Time: 0.00010722510700000004
